## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.73,74,11,16.35,few clouds
1,1,Solnechnyy,RU,50.7214,136.6319,12.67,95,100,3.18,light snow
2,2,Castro,BR,-24.7911,-50.0119,75.43,60,85,3.74,overcast clouds
3,3,Albany,US,42.6001,-73.9662,30.85,84,91,2.98,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,63.55,84,75,7.61,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.73,74,11,16.35,few clouds
2,2,Castro,BR,-24.7911,-50.0119,75.43,60,85,3.74,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,63.55,84,75,7.61,broken clouds
7,7,Sao Filipe,CV,14.8961,-24.4956,76.80,70,95,14.70,overcast clouds
9,9,Meulaboh,ID,4.1363,96.1285,74.34,91,100,4.52,moderate rain


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                354
City                   354
Country                351
Lat                    354
Lng                    354
Max Temp               354
Humidity               354
Cloudiness             354
Wind Speed             354
Current Description    354
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.73,few clouds,-23.1203,-134.9692,
2,Castro,BR,75.43,overcast clouds,-24.7911,-50.0119,
4,Hermanus,ZA,63.55,broken clouds,-34.4187,19.2345,
7,Sao Filipe,CV,76.80,overcast clouds,14.8961,-24.4956,
9,Meulaboh,ID,74.34,moderate rain,4.1363,96.1285,
10,Goianira,BR,80.42,broken clouds,-16.4961,-49.4264,
11,Lages,BR,71.51,overcast clouds,-27.8161,-50.3261,
12,Busselton,AU,61.63,scattered clouds,-33.6500,115.3333,
14,Saint George,US,77.47,clear sky,37.1041,-113.5841,
15,Vila Velha,BR,76.95,broken clouds,-20.3297,-40.2925,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.73,few clouds,-23.1203,-134.9692,People ThankYou
2,Castro,BR,75.43,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
4,Hermanus,ZA,63.55,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Sao Filipe,CV,76.80,overcast clouds,14.8961,-24.4956,Hotel Xaguate
9,Meulaboh,ID,74.34,moderate rain,4.1363,96.1285,Kost Pria
10,Goianira,BR,80.42,broken clouds,-16.4961,-49.4264,Estância Palmeiras
11,Lages,BR,71.51,overcast clouds,-27.8161,-50.3261,"Le CanarD Lages - Hotel em Lages, Excelente Lo..."
12,Busselton,AU,61.63,scattered clouds,-33.6500,115.3333,Observatory Guest House
14,Saint George,US,77.47,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
15,Vila Velha,BR,76.95,broken clouds,-20.3297,-40.2925,Quality Suites Vila Velha


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}  °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = clean_hotel_df[""]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))